## Imports

In [1]:
import numpy as np 
import pandas as pd 
import sqlite3 

## Make DataFrames

In [2]:
# Convert csvs to DataFrames
moons_df = pd.read_csv('../datasets/full_moons.csv')
plate_df = pd.read_csv('../datasets/plate_boundaries.csv')
quake_df = pd.read_csv('../datasets/significant_earthquakes.csv')

## Date/Time to Hours

In [3]:
# This is the value that all dates are compared to 
baseline = pd.to_datetime('1900-01-01 00:00:00')

For Moons DataFrame

In [4]:
# Combine 'Date' and 'Time' columns into 'DateTime' 
moons_df['DateTime'] = pd.to_datetime(moons_df['Date'] + ' ' + moons_df['Time'], format="%d %B %Y %I:%M:%S %p")

In [5]:
# Convert time elapsed since baseline into hours, then rounds
moons_df['time'] = (moons_df['DateTime'] - baseline).dt.total_seconds() / 3600
moons_df['time'] = moons_df['time'].round(decimals=2)

For Quakes DataFrame

In [6]:
# Convert time elapsed since baseline into hours, then rounds to int
quake_df['time'] = pd.to_datetime(quake_df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ")
quake_df['time'] = (quake_df['time'] - baseline).dt.total_seconds() / 3600
quake_df['time'] = quake_df['time'].round(decimals=0).astype(int)

## Column Cleaning

For Moons DataFrame

In [7]:
# Drop unneeded columns
moons_df.drop(columns=['Day', 'Date', 'Time', 'Flag', 'DateTime'], inplace=True)

In [8]:
# Create empty DataFrame to store interpolated new moon rows at midpoints 
new_moons_df = pd.DataFrame(columns=moons_df.columns)

# Loop through the rows of 'moons_df'
for i in range(len(moons_df) - 1):
    # Get the mean of each pair of full moon rows and adds it to 'new_moons_df'
    avg_time = (moons_df['time'].iloc[i] + moons_df['time'].iloc[i+1]) / 2
    row = pd.DataFrame([avg_time], columns=['time'])
    new_moons_df = pd.concat([new_moons_df, row], ignore_index=True)

# Combine the full moon rows with the new moon rows
moons_df = pd.concat([moons_df, new_moons_df]).sort_values('time').reset_index(drop=True)

In [9]:
# Round to int now that calculations are complete 
moons_df['time'] = moons_df['time'].round(decimals=0).astype(int)

In [10]:
# Change index to id 
moons_df.reset_index(level=0, inplace=True)
moons_df.rename(columns={'index': 'moon_ID'}, inplace=True)
moons_df.set_index('moon_ID', inplace=True)

For Plates DataFrame

In [11]:
# Change index to id 
plate_df.reset_index(level=0, inplace=True)
plate_df.rename(columns={'index': 'point_ID'}, inplace=True)
plate_df.set_index('point_ID', inplace=True)

For Quakes DataFrame

In [12]:
# Select earthquake-derived seismic activity only 
type_mask = quake_df['type'] == 'earthquake'
quake_df = quake_df[type_mask]

# Select quakes measured with moment magnitude scale only
mag_type_mask = quake_df['magType'] == 'mw'
quake_df = quake_df[mag_type_mask]

In [13]:
# Drop unneeded columns
quake_df.drop(columns=['Unnamed: 0','magType', 'nst', 'dmin','rms', 'net', 'id', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource', 'gap'], inplace=True)

In [14]:
# Change index to id 
quake_df.reset_index(level=0, inplace=True)
quake_df.rename(columns={'index': 'quake_ID'}, inplace=True)
quake_df.set_index('quake_ID', inplace=True)

## Remove null values

In [15]:
# Print amount of null values in each column 
print("MOONS DF", "\n", moons_df.isna().sum(), "\n") 
print("QUAKE DF", "\n", quake_df.isna().sum(), "\n")
print("PLATE DF", "\n", plate_df.isna().sum())

MOONS DF 
 time    0
dtype: int64 

QUAKE DF 
 time         0
latitude     0
longitude    0
depth        7
mag          0
dtype: int64 

PLATE DF 
 plate    0
lat      0
lon      0
dtype: int64


In [16]:
# Fill null values in 'depth' column of quake_df with 0 
quake_df['depth'].fillna(0, inplace=True)

## Aftershock/Preshock remover

In [17]:
def shock_identifier(quake_df):
    # Sort the dataframe by time, latitude, longitude, and magnitude
    quake_df.sort_values(['time', 'latitude', 'longitude', 'mag'], inplace=True)
    
    quake_df['shock_type'] = np.nan
    quake_df['hours_diff'] = np.nan

    for i in range(len(quake_df)):
        # Get the current earthquake
        cur_quake = quake_df.iloc[i]

        # Get all earthquakes within the given range of the current one
        in_range = quake_df[
            ((quake_df.time >= cur_quake.time - 2190) & (quake_df.time <= cur_quake.time + 2190)) & 
            (abs(quake_df.latitude - cur_quake.latitude) <= 17) & 
            (abs(quake_df.longitude - cur_quake.longitude) <= 17)
        ]

        # Get the quake with maximum magnitude within range
        max_mag_quake = in_range[in_range.mag == in_range.mag.max()]

        # If there's more than one quake with maximum magnitude, take the earliest one
        if len(max_mag_quake) > 1:
            max_mag_quake = max_mag_quake[max_mag_quake.time == max_mag_quake.time.min()]

        max_mag_quake_id = max_mag_quake.index[0]

        # If the current quake is the main quake
        if cur_quake.name == max_mag_quake_id:
            quake_df.at[cur_quake.name, 'shock_type'] = 'M'
            quake_df.at[cur_quake.name, 'hours_diff'] = 0
        else:
            quake_df.at[cur_quake.name, 'hours_diff'] = cur_quake.time - max_mag_quake.time.values[0]
            # If the current quake occurred before the main quake
            if quake_df.at[cur_quake.name, 'hours_diff'] < 0:
                quake_df.at[cur_quake.name, 'shock_type'] = 'P'
            else:  # If the current quake occurred after the main quake
                quake_df.at[cur_quake.name, 'shock_type'] = 'A'

    return quake_df

In [18]:
# Test the function 
shock_identifier(quake_df).head(5)

,time,latitude,longitude,depth,mag,shock_type,hours_diff
quake_ID,,,,,,,
0,6756,57.090,-153.480,0.0,7.86,M,0.0
16,37306,41.758,23.249,15.0,7.02,M,0.0
17,37306,41.802,23.108,15.0,6.84,A,0.0
19,39279,51.424,161.638,15.0,7.50,P,-6.0
18,39285,52.763,160.277,30.0,7.70,M,0.0


In [19]:
# Mask for preshocks, mainshocks and aftershocks
preshock_mask = quake_df["shock_type"] == 'P'
mainshock_mask = quake_df["shock_type"] == 'M'
aftershock_mask = quake_df["shock_type"] == 'A'

In [20]:
# Apply mask to quakes_df DataFrame
preshocks = quake_df[preshock_mask]
mainshocks = quake_df[mainshock_mask]
aftershocks = quake_df[aftershock_mask]

In [21]:
# Check proportions
print(len(preshocks))
print(len(mainshocks))
print(len(aftershocks))

8880
3706
11674


## Mainshock DataFrame

In [22]:
# Mask for quakes above mag 7 threshold
mains_df = mainshocks[mainshocks['mag'] > 7]
min(mains_df['mag'])

7.01

In [23]:
# Clean columns
mains_df.drop(columns=['depth', 'shock_type', 'hours_diff'], inplace=True)

/tmp/ipykernel_1834/3215635749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mains_df.drop(columns=['depth', 'shock_type', 'hours_diff'], inplace=True)


In [24]:
# Add columns for L1 & L2 distances
mains_df[['first_L1_dist', 'first_L2_dist', 'second_L1_dist', 'second_L2_dist', 'third_L1_dist', 'third_L2_dist']] = np.nan

/tmp/ipykernel_1834/3722229092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mains_df[['first_L1_dist', 'first_L2_dist', 'second_L1_dist', 'second_L2_dist', 'third_L1_dist', 'third_L2_dist']] = np.nan
/tmp/ipykernel_1834/3722229092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mains_df[['first_L1_dist', 'first_L2_dist', 'second_L1_dist', 'second_L2_dist', 'third_L1_dist', 'third_L2_dist']] = np.nan
/tmp/ipykernel_1834/3722229092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [25]:
# Check number of quakes and columns
print(len(mains_df))
print(list(mains_df.columns))

648
['time', 'latitude', 'longitude', 'mag', 'first_L1_dist', 'first_L2_dist', 'second_L1_dist', 'second_L2_dist', 'third_L1_dist', 'third_L2_dist']


In [26]:
mains_df.head(1)

,time,latitude,longitude,mag,first_L1_dist,first_L2_dist,second_L1_dist,second_L2_dist,third_L1_dist,third_L2_dist
quake_ID,,,,,,,,,,
0,6756,57.09,-153.48,7.86,NaN,NaN,NaN,NaN,NaN,NaN


## L1 & L2 distances

In [27]:
def distance_calculator(curr_quake, prev_quake):
    """
    Calculate the haversine and Manhattan distances between two points (specified in decimal degrees).
    Vectorized version for pandas df
    Computes distance in km
    """
    earth_radius = 6371
    
    curr_lat, curr_lon = curr_quake['latitude'], curr_quake['longitude']
    prev_lat, prev_lon = prev_quake['latitude'], prev_quake['longitude']

    curr_lat_rad, curr_lon_rad = np.radians(curr_lat), np.radians(curr_lon)
    prev_lat_rad, prev_lon_rad = np.radians(prev_lat), np.radians(prev_lon)

    diff_lat_rad = prev_lat_rad - curr_lat_rad
    diff_lon_rad = prev_lon_rad - curr_lon_rad
    
    manhattan_rad = np.abs(diff_lon_rad) + np.abs(diff_lat_rad)
    manhattan_km = manhattan_rad * earth_radius

    a = (np.sin(diff_lat_rad / 2.0)**2 + np.cos(curr_lat_rad) * np.cos(prev_lat_rad) * np.sin(diff_lon_rad / 2.0)**2)
    haversine_rad = 2 * np.arcsin(np.sqrt(a))
    haversine_km = haversine_rad * earth_radius

    return dict(
        haversine_in_km = haversine_km,
        manhattan_in_km = manhattan_km
    )

In [28]:
def threshold_applier(curr_quake, prev_quake_1, prev_quake_2, prev_quake_3, time_window):
    
    prev_dist_1, prev_dist_2, prev_dist_3 = None, None, None

    if prev_quake_1['time'] >= time_window:
        prev_dist_1 = distance_calculator(curr_quake, prev_quake_1)
        
    if prev_quake_2['time'] >= time_window:
        prev_dist_2 = distance_calculator(curr_quake, prev_quake_2)
    
    if prev_quake_3['time'] >= time_window:
        prev_dist_3 = distance_calculator(curr_quake, prev_quake_3)
        
    return prev_dist_1, prev_dist_2, prev_dist_3

In [29]:
def L1_and_L2_columns(mains_df):
    for i in range(len(mains_df)):
        # Handle first 3 rows
        if i < 3:
            continue
        
        curr_quake = mains_df.iloc[i]
        prev_quake_1 = mains_df.iloc[i-1]
        prev_quake_2 = mains_df.iloc[i-2]
        prev_quake_3 = mains_df.iloc[i-3]
        
        curr_time = curr_quake['time']
        time_window = curr_time - 1460
    
        prev_dist_1, prev_dist_2, prev_dist_3 = threshold_applier(curr_quake, prev_quake_1, prev_quake_2, prev_quake_3, time_window)
        
        # Populate the dataframe with distances
        mains_df.at[mains_df.index[i], 'first_L1_dist'] = prev_dist_1['manhattan_in_km'] if prev_dist_1 else np.nan
        mains_df.at[mains_df.index[i], 'first_L2_dist'] = prev_dist_1['haversine_in_km'] if prev_dist_1 else np.nan
        mains_df.at[mains_df.index[i], 'second_L1_dist'] = prev_dist_2['manhattan_in_km'] if prev_dist_2 else np.nan
        mains_df.at[mains_df.index[i], 'second_L2_dist'] = prev_dist_2['haversine_in_km'] if prev_dist_2 else np.nan
        mains_df.at[mains_df.index[i], 'third_L1_dist'] = prev_dist_3['manhattan_in_km'] if prev_dist_3 else np.nan
        mains_df.at[mains_df.index[i], 'third_L2_dist'] = prev_dist_3['haversine_in_km'] if prev_dist_3 else np.nan

    return mains_df

In [30]:
mains_df = L1_and_L2_columns(mains_df)
mains_df.sample(20)

,time,latitude,longitude,mag,first_L1_dist,first_L2_dist,second_L1_dist,second_L2_dist,third_L1_dist,third_L2_dist
quake_ID,,,,,,,,,,
11122,565577,-15.534,167.666,7.16,33574.307346,10716.108208,9152.343217,6686.349921,26254.234130,11898.063057
13796,612626,2.005,94.489,7.59,NaN,NaN,NaN,NaN,NaN,NaN
8498,488889,-5.062,152.855,7.31,NaN,NaN,NaN,NaN,NaN,NaN
51401,842749,-9.593,-79.587,7.50,23333.143407,17641.560063,NaN,NaN,NaN,NaN
1828,228831,12.292,-88.801,7.13,30070.777597,12603.620631,NaN,NaN,NaN,NaN
705,140984,29.510,131.875,7.41,6117.611279,4406.731764,NaN,NaN,NaN,NaN
8108,476196,29.112,-113.024,7.21,13041.830556,9620.108631,NaN,NaN,NaN,NaN
1696,221555,-34.565,58.728,7.04,13974.088821,10533.605517,NaN,NaN,NaN,NaN
7898,467609,-37.093,-72.866,7.55,28730.322757,13404.588267,6955.465051,5789.975084,19768.456449,13345.563311


In [31]:
# Prints how 
print("QUAKE DF", "\n", len(mains_df) - mains_df.isna().sum(), "\n")

QUAKE DF 
 time              648
latitude          648
longitude         648
mag               648
first_L1_dist     453
first_L2_dist     453
second_L1_dist    214
second_L2_dist    214
third_L1_dist      69
third_L2_dist      69
dtype: int64 



## Store as SQL

In [32]:
# Creates and connects to repo SQL database
conn = sqlite3.connect('../datasets/database.db')

In [33]:
# Creates SQL tables from each DataFrame
moons_table_name = 'moons_table'
moons_df.to_sql(moons_table_name, conn, if_exists='replace', index=True)

plate_table_name = 'plate_table'
plate_df.to_sql(plate_table_name, conn, if_exists='replace', index=True)

quake_table_name = 'quake_table'
quake_df.to_sql(quake_table_name, conn, if_exists='replace', index=True)

648

## Add moon intervals

In [35]:
c = conn.cursor()

In [41]:
def last_moon(quake_time):
    query = f"""
        SELECT time 
        FROM moons_table
        WHERE time < {quake_time}
        ORDER BY time DESC 
        LIMIT 1 
      """
    
    c.execute(query)
    last_moon_time = c.fetchone()[0]
    
    return last_moon_time

last_moon(6756), last_moon(37306)

(6734, 37214)

In [42]:
def next_moon(quake_time):
    query = f"""
        SELECT time
        FROM moons_table
        WHERE time > {quake_time}
        ORDER BY time ASC 
        LIMIT 1 
      """
    c.execute(query)
    next_moon_time = c.fetchone()[0]
    
    return next_moon_time

next_moon(6756), next_moon(37306)

(7087, 37567)

In [43]:
mains_df['time_since_last_moon'] = mains_df['time'].apply(lambda x: x - last_moon(x))
mains_df['time_to_next_moon'] = mains_df['time'].apply(lambda x: next_moon(x)-x)

/tmp/ipykernel_1834/3034946508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mains_df['time_since_last_moon'] = mains_df['time'].apply(lambda x: x - last_moon(x))
/tmp/ipykernel_1834/3034946508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mains_df['time_to_next_moon'] = mains_df['time'].apply(lambda x: next_moon(x)-x)


In [44]:
mains_df['time_to_nearest_moon'] = mains_df[["time_since_last_moon", "time_to_next_moon"]].apply(lambda row: min(row["time_since_last_moon"], row["time_to_next_moon"]), axis = 1)

/tmp/ipykernel_1834/2175792247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mains_df['time_to_nearest_moon'] = mains_df[["time_since_last_moon", "time_to_next_moon"]].apply(lambda row: min(row["time_since_last_moon"], row["time_to_next_moon"]), axis = 1)


In [48]:
mains_df.sample(15)

,time,latitude,longitude,mag,first_L1_dist,first_L2_dist,second_L1_dist,second_L2_dist,third_L1_dist,third_L2_dist,time_since_last_moon,time_to_next_moon,time_to_nearest_moon
quake_ID,,,,,,,,,,,,,
2641,253908,6.707,124.245,7.42,26498.084604,16436.168162,NaN,NaN,NaN,NaN,176,177,176
1231,183556,-39.102,-73.462,7.29,34214.122954,16435.367826,NaN,NaN,NaN,NaN,337,16,16
4716,322917,38.279,142.235,7.25,8812.753911,5931.540379,NaN,NaN,NaN,NaN,86,267,86
7432,454915,23.092,121.214,7.81,4216.178034,3084.079728,6342.669811,3966.114576,NaN,NaN,266,91,91
263,75682,31.209,87.679,7.04,25539.806726,16231.044076,NaN,NaN,NaN,NaN,196,156,156
532,119057,48.108,156.059,7.03,44328.413481,8216.747777,NaN,NaN,NaN,NaN,346,9,9
42406,792266,5.121,32.145,7.20,17081.542241,10984.388256,10524.266222,10074.610053,13949.959522,12559.319755,244,111,111
31838,735414,-7.481,128.168,7.30,22995.444415,12093.910960,24216.587099,15726.844469,NaN,NaN,89,266,89
1718,222089,1.223,126.303,7.13,2865.493260,2355.011420,11493.441203,8053.275971,6919.104310,5122.625844,235,118,118


In [47]:
mains_table_name = 'mains_table'
mains_df.to_sql(mains_table_name, conn, if_exists='replace', index=True)

648